# Module 05: DataFrame Operations

**Difficulty**: ⭐⭐

**Estimated Time**: 80-95 minutes

**Prerequisites**: 
- Module 00: Introduction to Big Data and Spark Ecosystem
- Module 01: PySpark Setup and SparkSession
- Module 03: DataFrames and Datasets
- Module 04: Data Loading and Saving

## Learning Objectives

By the end of this notebook, you will be able to:
1. Filter and select data using various methods (filter, where, select, selectExpr)
2. Perform grouping and aggregation operations (groupBy, agg, pivot)
3. Join DataFrames using different join types (inner, left, right, full, cross)
4. Sort and order data efficiently (orderBy, sort)
5. Use built-in functions for data transformation and manipulation

## Setup

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from pyspark.sql.functions import (
    col, lit, when, expr, concat, concat_ws,
    sum, avg, count, min, max, countDistinct,
    round, upper, lower, trim, substring,
    year, month, dayofmonth, current_date,
    datediff, date_add, date_sub
)
from datetime import date
import random

# Create SparkSession
spark = SparkSession.builder \
    .appName("Module 05: DataFrame Operations") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print(f"✓ SparkSession created: {spark.sparkContext.appName}")
print(f"  Spark version: {spark.version}")
print(f"  Spark UI: {spark.sparkContext.uiWebUrl}")

### Create Sample DataFrames

In [ ]:
# Sample employees data
employees_data = [
    (1, "Alice Johnson", "Engineering", 75000, date(2020, 1, 15)),
    (2, "Bob Smith", "Sales", 65000, date(2019, 6, 10)),
    (3, "Charlie Brown", "Engineering", 80000, date(2021, 3, 22)),
    (4, "Diana Prince", "Marketing", 70000, date(2020, 8, 5)),
    (5, "Eve Davis", "Engineering", 85000, date(2018, 11, 30)),
    (6, "Frank Miller", "Sales", 68000, date(2022, 2, 14)),
    (7, "Grace Lee", "Marketing", 72000, date(2021, 9, 1)),
    (8, "Henry Wilson", "Engineering", 78000, date(2019, 4, 20)),
    (9, "Ivy Chen", "Sales", 71000, date(2020, 12, 10)),
    (10, "Jack Thompson", "Engineering", 82000, date(2021, 7, 5))
]

df_employees = spark.createDataFrame(
    employees_data,
    ["id", "name", "department", "salary", "hire_date"]
)

print("Employees DataFrame:")
df_employees.show()

# Sample departments data (for joins)
departments_data = [
    ("Engineering", "Building A", "Alice Johnson"),
    ("Sales", "Building B", "Bob Smith"),
    ("Marketing", "Building C", "Diana Prince"),
    ("HR", "Building A", "Unknown")  # No employees in HR
]

df_departments = spark.createDataFrame(
    departments_data,
    ["dept_name", "location", "manager"]
)

print("\nDepartments DataFrame:")
df_departments.show()

# Sample sales data
sales_data = []
products = ['Laptop', 'Phone', 'Tablet', 'Monitor', 'Keyboard']
regions = ['North', 'South', 'East', 'West']

for i in range(100):
    sales_data.append((
        i + 1,
        random.choice(products),
        random.choice(regions),
        random.randint(1, 10),
        round(random.uniform(100, 2000), 2)
    ))

df_sales = spark.createDataFrame(
    sales_data,
    ["sale_id", "product", "region", "quantity", "revenue"]
)

print("\nSales DataFrame (first 10 rows):")
df_sales.show(10)

## 1. Filtering and Selecting Data

### Filtering with filter() and where()

**Note**: `filter()` and `where()` are aliases - they do the same thing!

In [ ]:
# Method 1: String expression
print("Filter using string expression:")
df_employees.filter("salary > 75000").show()

# Method 2: Column object
print("\nFilter using Column object:")
df_employees.filter(col("salary") > 75000).show()

# Method 3: where() - same as filter()
print("\nFilter using where():")
df_employees.where(col("department") == "Engineering").show()

In [ ]:
# Multiple conditions with AND
print("Multiple conditions (AND):")
df_employees.filter(
    (col("department") == "Engineering") & (col("salary") > 75000)
).show()

# Multiple conditions with OR
print("\nMultiple conditions (OR):")
df_employees.filter(
    (col("department") == "Engineering") | (col("department") == "Sales")
).show()

# NOT condition
print("\nNOT condition:")
df_employees.filter(~(col("department") == "Engineering")).show()

# ⚠️ Important: Use & and | (not 'and' and 'or') and use parentheses!

In [ ]:
# Filter with IN
print("Filter with IN:")
df_employees.filter(col("department").isin("Engineering", "Sales")).show()

# Filter with LIKE (pattern matching)
print("\nFilter with LIKE:")
df_employees.filter(col("name").like("%Johnson%")).show()

# Filter with BETWEEN
print("\nFilter with BETWEEN:")
df_employees.filter(col("salary").between(70000, 80000)).show()

### Selecting Columns

In [ ]:
# Select specific columns
print("Select specific columns:")
df_employees.select("name", "department", "salary").show(5)

# Select with transformations
print("\nSelect with transformations:")
df_employees.select(
    col("name"),
    (col("salary") * 12).alias("annual_salary")
).show(5)

# Select with multiple transformations
print("\nSelect with multiple transformations:")
df_employees.select(
    col("name"),
    col("salary"),
    (col("salary") * 1.10).alias("salary_with_raise"),
    ((col("salary") * 1.10) - col("salary")).alias("raise_amount")
).show(5)

In [ ]:
# selectExpr() - Use SQL expressions
print("selectExpr() - SQL expressions:")
df_employees.selectExpr(
    "name",
    "salary",
    "salary * 12 as annual_salary",
    "salary * 0.10 as bonus"
).show(5)

# expr() - Use expressions inline
print("\nexpr() - Inline expressions:")
df_employees.select(
    col("name"),
    expr("salary * 12").alias("annual_salary")
).show(5)

### Conditional Columns with when()

In [ ]:
# Simple when/otherwise (if/else)
print("Simple when/otherwise:")
df_employees.select(
    col("name"),
    col("salary"),
    when(col("salary") > 75000, "High")
        .otherwise("Standard")
        .alias("salary_tier")
).show()

# Multiple conditions (if/elif/else)
print("\nMultiple conditions:")
df_employees.select(
    col("name"),
    col("salary"),
    when(col("salary") >= 80000, "Senior")
        .when(col("salary") >= 70000, "Mid-Level")
        .otherwise("Junior")
        .alias("level")
).show()

## 2. Sorting and Ordering

### orderBy() and sort()

**Note**: `orderBy()` and `sort()` are aliases - they do the same thing!

In [ ]:
# Sort ascending (default)
print("Sort by salary (ascending):")
df_employees.orderBy("salary").show()

# Sort descending
print("\nSort by salary (descending):")
df_employees.orderBy(col("salary").desc()).show()

# Alternative syntax for descending
print("\nAlternative descending syntax:")
df_employees.orderBy("salary", ascending=False).show(5)

In [ ]:
# Sort by multiple columns
print("Sort by department (asc), then salary (desc):")
df_employees.orderBy(
    col("department").asc(),
    col("salary").desc()
).show()

## 3. Grouping and Aggregation

### Basic groupBy()

In [ ]:
# Count by department
print("Count employees by department:")
df_employees.groupBy("department").count().show()

# Average salary by department
print("\nAverage salary by department:")
df_employees.groupBy("department") \
    .avg("salary") \
    .withColumnRenamed("avg(salary)", "avg_salary") \
    .show()

# Multiple aggregations
print("\nMultiple aggregations:")
df_employees.groupBy("department") \
    .agg(
        count("*").alias("num_employees"),
        avg("salary").alias("avg_salary"),
        min("salary").alias("min_salary"),
        max("salary").alias("max_salary")
    ) \
    .show()

In [ ]:
# Round numeric results
print("Aggregations with rounding:")
df_employees.groupBy("department") \
    .agg(
        count("*").alias("count"),
        round(avg("salary"), 2).alias("avg_salary")
    ) \
    .orderBy("avg_salary", ascending=False) \
    .show()

### Advanced Aggregations

In [ ]:
# Sales analysis
print("Sales by product:")
df_sales.groupBy("product") \
    .agg(
        count("*").alias("num_sales"),
        sum("quantity").alias("total_quantity"),
        round(sum("revenue"), 2).alias("total_revenue"),
        round(avg("revenue"), 2).alias("avg_revenue")
    ) \
    .orderBy("total_revenue", ascending=False) \
    .show()

# Sales by region and product
print("\nSales by region and product:")
df_sales.groupBy("region", "product") \
    .agg(
        count("*").alias("num_sales"),
        round(sum("revenue"), 2).alias("total_revenue")
    ) \
    .orderBy("region", "total_revenue", ascending=[True, False]) \
    .show(10)

### Pivot Tables

In [ ]:
# Create pivot table: regions as rows, products as columns
print("Pivot: Total revenue by region and product:")
df_sales.groupBy("region") \
    .pivot("product") \
    .agg(round(sum("revenue"), 2)) \
    .show()

# Pivot with multiple aggregations
print("\nPivot: Count of sales by region and product:")
df_sales.groupBy("region") \
    .pivot("product") \
    .count() \
    .fillna(0) \
    .show()

## 4. Joining DataFrames

### Join Types

| Join Type | Description | Result |
|-----------|-------------|--------|
| `inner` (default) | Only matching rows from both | Intersection |
| `left` / `left_outer` | All from left + matching from right | Left + matches |
| `right` / `right_outer` | All from right + matching from left | Right + matches |
| `full` / `full_outer` | All rows from both | Union |
| `cross` | Cartesian product | All combinations |
| `left_semi` | Left rows with match in right | Like inner, but only left columns |
| `left_anti` | Left rows WITHOUT match in right | Opposite of left_semi |

### Inner Join

In [ ]:
# Inner join (default)
print("Inner join:")
df_joined = df_employees.join(
    df_departments,
    df_employees["department"] == df_departments["dept_name"],
    "inner"
)
df_joined.select("name", "department", "location", "manager").show()

# Note: HR department not shown (no employees)

### Left Join

In [ ]:
# Left join - all employees + matching departments
print("Left join:")
df_left = df_employees.join(
    df_departments,
    df_employees["department"] == df_departments["dept_name"],
    "left"
)
df_left.select("name", "department", "location").show()

# All employees included, even if no matching department

### Right Join

In [ ]:
# Right join - all departments + matching employees
print("Right join:")
df_right = df_employees.join(
    df_departments,
    df_employees["department"] == df_departments["dept_name"],
    "right"
)
df_right.select("name", "dept_name", "location").show()

# HR department included (with null for employee columns)

### Full Outer Join

In [ ]:
# Full outer join - all rows from both
print("Full outer join:")
df_full = df_employees.join(
    df_departments,
    df_employees["department"] == df_departments["dept_name"],
    "full"
)
df_full.select("name", "department", "dept_name", "location").show()

# Both employees without departments AND departments without employees

### Semi and Anti Joins

In [ ]:
# Left semi join - employees with matching department
print("Left semi join (employees in departments table):")
df_semi = df_employees.join(
    df_departments,
    df_employees["department"] == df_departments["dept_name"],
    "left_semi"
)
df_semi.show()

# Left anti join - employees WITHOUT matching department
print("\nLeft anti join (employees NOT in departments table):")
df_anti = df_employees.join(
    df_departments,
    df_employees["department"] == df_departments["dept_name"],
    "left_anti"
)
df_anti.show()

print("\n💡 Semi/anti joins are useful for filtering based on existence in another table")

### Join on Multiple Columns

In [ ]:
# When column names match, you can use simpler syntax
df_departments2 = df_departments.withColumnRenamed("dept_name", "department")

print("Join on matching column name:")
df_simple = df_employees.join(df_departments2, "department", "inner")
df_simple.select("name", "department", "location").show()

# Multiple columns
print("\nJoin on multiple columns:")
# df1.join(df2, ["col1", "col2"], "inner")

## 5. String Functions

In [ ]:
# String manipulation
print("String functions:")
df_employees.select(
    col("name"),
    upper(col("name")).alias("upper_name"),
    lower(col("name")).alias("lower_name"),
    substring(col("name"), 1, 5).alias("first_5_chars"),
    concat(col("name"), lit(" - "), col("department")).alias("name_dept")
).show(5, truncate=False)

## 6. Date Functions

In [ ]:
# Date manipulation
print("Date functions:")
df_employees.select(
    col("name"),
    col("hire_date"),
    year(col("hire_date")).alias("hire_year"),
    month(col("hire_date")).alias("hire_month"),
    datediff(current_date(), col("hire_date")).alias("days_employed"),
    date_add(col("hire_date"), 365).alias("one_year_anniversary")
).show(5)

## Exercises

### Exercise 1: Filtering and Selection

Using the `df_employees` DataFrame:
1. Filter employees hired after 2020-01-01
2. Select only their name, department, and hire_date
3. Add a column showing years since hire (use year and current_date)
4. Sort by hire date (most recent first)

In [ ]:
# Exercise 1: Your code here

# Your code here

### Exercise 2: Conditional Logic

Create a salary analysis report showing:
- Employee name and salary
- A "bonus_percentage" column: 15% for salary >= 80000, 10% for >= 70000, 5% otherwise
- A "bonus_amount" column (salary * bonus_percentage)
- A "total_compensation" column (salary + bonus_amount)

Sort by total_compensation descending.

In [ ]:
# Exercise 2: Your code here

# Your code here

### Exercise 3: Aggregation

Using the `df_sales` DataFrame, create a report showing:
1. Total revenue by region
2. Average revenue per sale by region
3. Number of sales by region
4. Sort by total revenue (descending)
5. Round all monetary values to 2 decimal places

In [ ]:
# Exercise 3: Your code here

# Your code here

### Exercise 4: Joins and Aggregation

1. Join `df_employees` with `df_departments` (use appropriate join type)
2. Group by department location
3. Calculate:
   - Number of employees per location
   - Average salary per location
   - Total payroll per location (sum of salaries)
4. Sort by total payroll (descending)

In [ ]:
# Exercise 4: Your code here

# Your code here

## Solutions

### Exercise 1 Solution

In [ ]:
# Solution 1: Filtering and Selection

from pyspark.sql.functions import year, current_date

df_recent = df_employees \
    .filter(col("hire_date") > "2020-01-01") \
    .select(
        col("name"),
        col("department"),
        col("hire_date"),
        (year(current_date()) - year(col("hire_date"))).alias("years_employed")
    ) \
    .orderBy(col("hire_date").desc())

print("Employees hired after 2020-01-01:")
df_recent.show()

### Exercise 2 Solution

In [ ]:
# Solution 2: Conditional Logic

df_bonus = df_employees.select(
    col("name"),
    col("salary"),
    when(col("salary") >= 80000, 0.15)
        .when(col("salary") >= 70000, 0.10)
        .otherwise(0.05)
        .alias("bonus_percentage"),
) \
.withColumn("bonus_amount", round(col("salary") * col("bonus_percentage"), 2)) \
.withColumn("total_compensation", col("salary") + col("bonus_amount")) \
.orderBy(col("total_compensation").desc())

print("Salary analysis with bonuses:")
df_bonus.show()

### Exercise 3 Solution

In [ ]:
# Solution 3: Aggregation

df_region_analysis = df_sales.groupBy("region") \
    .agg(
        count("*").alias("num_sales"),
        round(sum("revenue"), 2).alias("total_revenue"),
        round(avg("revenue"), 2).alias("avg_revenue_per_sale")
    ) \
    .orderBy(col("total_revenue").desc())

print("Sales analysis by region:")
df_region_analysis.show()

### Exercise 4 Solution

In [ ]:
# Solution 4: Joins and Aggregation

df_location_analysis = df_employees.join(
    df_departments,
    df_employees["department"] == df_departments["dept_name"],
    "left"  # Use left to include all employees
) \
.groupBy("location") \
.agg(
    count("*").alias("num_employees"),
    round(avg("salary"), 2).alias("avg_salary"),
    sum("salary").alias("total_payroll")
) \
.orderBy(col("total_payroll").desc())

print("Employee and payroll analysis by location:")
df_location_analysis.show()

## Summary

### Key Concepts Covered

✅ **Filtering**: Use `filter()` or `where()` with conditions

✅ **Selecting**: Use `select()`, `selectExpr()`, and transformations

✅ **Conditional Logic**: Use `when()` for if/else logic

✅ **Sorting**: Use `orderBy()` or `sort()` with `asc()` and `desc()`

✅ **Grouping**: Use `groupBy()` with aggregation functions

✅ **Joining**: Multiple join types for combining DataFrames

### Important Operations Summary

**Filtering**:
- `df.filter(condition)` / `df.where(condition)`
- Conditions: `==`, `!=`, `>`, `<`, `>=`, `<=`
- Logical: `&` (and), `|` (or), `~` (not)
- Special: `.isin()`, `.like()`, `.between()`

**Selecting**:
- `df.select(cols)`
- `df.selectExpr(sql_expressions)`
- `col.alias(new_name)` to rename

**Aggregating**:
- `df.groupBy(cols).agg(functions)`
- Functions: `count()`, `sum()`, `avg()`, `min()`, `max()`
- `round()` for numeric precision

**Joining**:
- `df1.join(df2, condition, join_type)`
- Types: `inner`, `left`, `right`, `full`, `cross`, `left_semi`, `left_anti`

### Common Functions

**String**: `upper()`, `lower()`, `trim()`, `substring()`, `concat()`

**Date**: `year()`, `month()`, `day()`, `datediff()`, `date_add()`, `date_sub()`

**Math**: `round()`, `abs()`, `ceil()`, `floor()`

**Conditional**: `when()`, `otherwise()`

### Best Practices

1. **Use parentheses** with multiple conditions: `(cond1) & (cond2)`
2. **Use `&` and `|`**, not `and` and `or`
3. **Alias columns** for readability: `.alias("new_name")`
4. **Chain operations** for clarity
5. **Round numbers** in final output for presentation
6. **Choose appropriate join type** based on your needs
7. **Use `agg()`** for multiple aggregations instead of chaining

### Performance Tips

- **Filter early**: Reduce data size before joins/aggregations
- **Select only needed columns**: Don't carry unnecessary data
- **Use broadcast joins** for small tables (covered in advanced modules)
- **Avoid UDFs** when built-in functions exist (covered in Module 07)

### What's Next?

In **Module 06: Spark SQL and Temporary Views**, you will:
- Write SQL queries on DataFrames
- Create temporary and global views
- Use the Spark SQL catalog
- Mix DataFrame API and SQL seamlessly

### Additional Resources

- [PySpark Functions API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html)
- [DataFrame Operations Guide](https://spark.apache.org/docs/latest/sql-ref.html)
- [Join Strategies](https://spark.apache.org/docs/latest/sql-performance-tuning.html#join-strategy-hints)

In [ ]:
# Cleanup
spark.stop()
print("SparkSession stopped. ✓")